In [33]:
import pandas as pd
from influxdb_client import InfluxDBClient, QueryApi
import os
import statsmodels.api as sm
import numpy as np

INFLUXDB_TOKEN = os.getenv("INFLUXDB_TOKEN")
INFLUXDB_ORG = os.getenv("INFLUXDB_ORG")
INFLUXDB_BUCKET = os.getenv("INFLUXDB_BUCKET")
INFLUXDB_URL = os.getenv("INFLUXDB_URL")

# Connect to InfluxDB
client = InfluxDBClient(url="http://localhost:8086", token=INFLUXDB_TOKEN, org=INFLUXDB_ORG, timeout=30_000)

query_api = client.query_api()
query = 'from(bucket: "crypto")\
  |> range(start: -1y)\
  |> filter(fn: (r) => r["_measurement"] == "BTCUSDT")'
result = query_api.query(org=INFLUXDB_ORG, query=query)




In [34]:
print(INFLUXDB_TOKEN)

0TUdHBu-5FvolnwzsE_xJDNw7Jsrv1W38o-8ImRSJPAiUjnczWcFfZOBgXLyrKXc4Hv1-KNcFWXYqPC-WYoVxA==


In [29]:
results = []
for table in result:
  for record in table.records:
    results.append((record.get_field(), record.get_value(),record.get_time()))
    
data=pd.DataFrame(results)
data=data.rename({0:"_field",1:"_value",2:"_time"},axis=1)
data['_time'] = pd.to_datetime(data['_time'],format='%Y-%m-%dT%H:%M:%SZ',errors='coerce')
data['_value']=pd.to_numeric(data['_value'],errors='coerce',downcast="float")
data.dropna(inplace=True)
data=pd.pivot(data,index='_time',columns='_field',values='_value')
data=data.reset_index()
data

_field,_time,close,high,kline_close_time,low,number_of_trades,open,quote_asset_volume,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,volume
0,2022-02-02 19:00:00+00:00,37566.56,37674.34,1.643832e+12,37401.75,31668.0,37486.41,4.379007e+07,610.37370,2.291880e+07,1166.16549
1,2022-02-02 20:00:00+00:00,37566.00,37662.14,1.643836e+12,37400.17,35773.0,37566.56,5.553320e+07,748.81467,2.810585e+07,1479.58286
2,2022-02-02 21:00:00+00:00,36922.51,37612.66,1.643839e+12,36586.95,72460.0,37566.00,1.276321e+08,1459.93033,5.396452e+07,3452.00636
3,2022-02-02 22:00:00+00:00,37006.93,37211.00,1.643843e+12,36820.50,42615.0,36922.51,5.692172e+07,735.68159,2.721524e+07,1538.78106
4,2022-02-02 23:00:00+00:00,36896.36,37133.16,1.643846e+12,36797.18,34402.0,37006.93,5.630153e+07,684.03642,2.527437e+07,1524.19620
...,...,...,...,...,...,...,...,...,...,...,...
8761,2023-02-02 20:00:00+00:00,23872.75,23878.00,1.675372e+12,23748.55,420584.0,23802.57,3.779602e+08,7948.75853,1.891977e+08,15880.07072
8762,2023-02-02 21:00:00+00:00,23455.65,23924.81,1.675375e+12,23413.02,574069.0,23872.76,5.892377e+08,11973.57629,2.840018e+08,24846.98228
8763,2023-02-02 22:00:00+00:00,23556.13,23618.98,1.675379e+12,23363.27,385382.0,23455.65,3.177552e+08,6693.68472,1.574665e+08,13508.07482
8764,2023-02-02 23:00:00+00:00,23488.94,23595.50,1.675382e+12,23423.75,294877.0,23556.70,2.513132e+08,5212.11185,1.225740e+08,10686.49872


In [30]:
exog=data.copy()
exog.drop('high',axis=1,inplace=True)

exog['_time']=pd.DatetimeIndex(exog.reset_index()['_time']) + pd.DateOffset(hours=1)
data=data[1:]
last_exog=exog.iloc[-1]
exog=exog.iloc[:-1]

data.set_index('_time',inplace=True)
exog.set_index('_time',inplace=True)

endog=data['high'].values.reshape(-1,1)

In [31]:
model = sm.tsa.arima.ARIMA(endog=endog[:-1],exog=np.array(exog[:-1],dtype='float'), order=(5,3,3))
results = model.fit(method='statespace')
results.forecast(steps=1,exog=exog.iloc[-1])

/Users/dariodm/opt/anaconda3/envs/proj_nuovo/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/dariodm/opt/anaconda3/envs/proj_nuovo/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


array([23228.02679103])